In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [2]:
url_inicial = 'https://abm.ojs.inecol.mx/index.php/abm/issue/archive'
url_root = 'https://abm.ojs.inecol.mx/index.php/abm/issue/archive'
r=requests.get(url_inicial)

In [3]:
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
box = soup.find('div', class_='flex_container issues_list')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://abm.ojs.inecol.mx/index.php/abm/issue/view/211',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/210',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/208',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/205',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/203',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/201',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/200',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/199',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/198',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/197',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/196',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/195',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/194',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/193',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/192',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/191',
 'https://abm.ojs.inecol.mx/index.php/abm/issue/view/190

In [5]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='section_content') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [6]:
vol2

['https://abm.ojs.inecol.mx/index.php/abm/article/view/2063',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1967',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1920',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1987',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1991',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1903',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1927',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2008',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1962',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2014',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2007',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/1989',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2025',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2024',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2049',
 'https://abm.ojs.inecol.mx/index.php/abm/article/view/2023',
 'https:

In [7]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='section_content') 
    volumen1=box1.findAll('article', class_='article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [8]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://abm.ojs.inecol.mx/index.php/abm/issue/archive


In [9]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

312

In [10]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [14]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Acta Botánica Mexicana'
    a='II. Biología y Química'
    tem='Botánica'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title article-full-title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='abstract').find('p').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='btn galley-link obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [15]:
list_scraper=list_scraper[0:312]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [16]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Acta Botánica Mexicana,II. Biología y Química,Botánica,Riqueza específica y distribución de licofitas...,Antecedentes y Objetivos:En la Sierra Norte de...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
1,Acta Botánica Mexicana,II. Biología y Química,Botánica,Reconocimiento de dos variedades taxonómicas d...,Antecedentes y Objetivos:Pyrodiniumes un géner...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
2,Acta Botánica Mexicana,II. Biología y Química,Botánica,Actividad antibacteriana y perfil fenólico del...,Antecedentes y Objetivos:Hyptis suaveolenses u...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
3,Acta Botánica Mexicana,II. Biología y Química,Botánica,"Primer registro de Ulex europaeus (Fabaceae, F...",Antecedentes y Objetivos:La especieUlex europa...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
4,Acta Botánica Mexicana,II. Biología y Química,Botánica,Comparación del contenido de betalaínas y azúc...,Antecedentes y Objetivos:Las betalaínas son pi...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
...,...,...,...,...,...,...,...,...
307,Acta Botánica Mexicana,II. Biología y Química,Botánica,Comparación anatómica e histoquímica de la mad...,El guayacán como planta medicinal se recomiend...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
308,Acta Botánica Mexicana,II. Biología y Química,Botánica,Nuevos registros de Basidiomycota para Jalisco...,Se reconocen 23 registros nuevos para la micob...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
309,Acta Botánica Mexicana,II. Biología y Química,Botánica,Los fenoles del polen del género Zea,Se analizó la variabilidad de la composición f...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...
310,Acta Botánica Mexicana,II. Biología y Química,Botánica,Hongos ascomicetos del bosque mesófilo de mont...,Se presenta una puesta al día de la diversidad...,None,https://abm.ojs.inecol.mx/index.php/abm/articl...,https://abm.ojs.inecol.mx/index.php/abm/articl...


In [17]:
df_catalogo.to_csv('Revista29.01.csv', index=False)